### 1. Import libraries and load data

In [ ]:
import numpy as np
import pandas as pd
import pybnesian as pbn
import sys
import statistics
import math
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from tqdm import tqdm
import matplotlib.pyplot as plt
from importlib import reload
import os
from datetime import datetime

import functions
functions = reload(functions)

(x_train, x_test) = # Load the training and testing datasets as Pandas DataFrames (each row makes for a sample, each column is an attribute)
                    # Name of the class variable must be 'class'

### 2. Parameters

In [ ]:
# Size of initial training batch
original_len = len(x_train)
max_len = math.floor(original_len)
ini_train_size = max_len

# Number of noise nodes 
noise_dim = 6

# Minimum allowed value
zero = sys.float_info.min

# Mean and variance for noise from which samples will be generated
sample_mean = 0
sample_variance = 5

# Size of samples used in tests (visualization)
test_sample_size = 100

# Size of each training sample (generation of best samples)
generation_size = 100

# Max number of training cycles
epochs = 20

# Logl score parameters: threshold and repulsion/attraction factor
logl_threshold = 0
logl_rep_attr = 1.1

# Size of the training batch for the reverse generator
rgen_train_size = 10000

# Mean and variance for noise from which rgen will be trained
sample_mean_rgen = 0
sample_variance_rgen = sample_variance

# Number of noise-sampling iterations to calculate mean anomaly score
mean_iter = 50

# Anomaly score distance power (1=Manhattan, 2=Euclidean...)
ano_power = 2

### 3. Net initialization

##### 3.1. Discriminator

In [ ]:
# Discriminator structure learning and fitting
disc_train_data = x_train.sample(ini_train_size)

# # Greedy hill-climbing
# disc_bn = pbn.hc(disc_train_data, bn_type=pbn.GaussianNetworkType())

# PC algorithm
learn_algo = pbn.PC()
hypothesis = pbn.LinearCorrelation(x_train)
disc_graph = learn_algo.estimate(hypot_test = hypothesis, allow_bidirected = False)
disc_dag = disc_graph.to_approximate_dag()
disc_bn = pbn.GaussianNetwork(disc_dag)
disc_bn.fit(x_train)

In [ ]:
# Base parameters and structure
base_nodes = disc_bn.nodes()
base_arcs = disc_bn.arcs()
base_cpds = []
for node in base_nodes:
  base_cpds.append(disc_bn.cpd(node))

##### 3.1.1. Discriminator pretest

In [ ]:
y_true = np.array(x_test['class'])
y_score = -1*disc_bn.logl(x_test)

fpr, tpr, thresholds = metrics.roc_curve(y_true, y_score)
roc_auc = metrics.auc(fpr, tpr)

print('Area under the ROC curve: ' + str(roc_auc))
display = metrics.RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc, estimator_name='Disc-Logl score')
display.plot()
# plt.savefig(model_dir+'pretestroc.png')
plt.show()

##### 3.2. Generator

In [ ]:
# Noise nodes
noise_nodes = []
for i in range(noise_dim):
  noise_nodes.append('noise'+str(i+1))
gen_nodes = base_nodes + noise_nodes

# Noise arcs
noise_arcs = []
for nnode in noise_nodes:
  for bnode in base_nodes:
    noise_arcs.append((nnode,bnode))

gen_arcs = noise_arcs
gen_bn = functions.reset_gen(noise_nodes, base_nodes, gen_arcs)
init_cpds = []

# Banned arcs
banned_arcs = []
for bnode in base_nodes:
  for nnode in noise_nodes:
    banned_arcs.append((bnode,nnode))

banned_edges = []
for bnode in base_nodes:
  for node in base_nodes:
    banned_edges.append((bnode,node))

# Adding only random normal noise arcs
for node in base_nodes:
  noise_parents = noise_nodes
  parents = noise_parents
  random_betas = []
  for i in range(len(noise_parents)):
    randnum = np.random.uniform(0,1,1)
    if randnum > 0.5:
      multiplier = -1
    else: 
      multiplier = 1
    random_betas.append(multiplier * np.random.normal(0.5,0.2))
  betas = [0] + random_betas
  variance = zero
  init_cpds.append(pbn.LinearGaussianCPD(node, parents, betas, variance))

# Introducing CPDs into generator
gen_bn.add_cpds(init_cpds)
gen_bn.fitted()

# Initial CPDs
print('Initial CPDs:')
for node in base_nodes:
  print(gen_bn.cpd(node))

### 4. Training

In [ ]:
logl_array = []
logl_mean_prev = 0

pbar = tqdm(total=epochs, position=0, leave=True)

for n in range(epochs):
  
  # Generates batch of samples
  batch_samples = functions.gen_samples_genetically(gen_bn, disc_bn, generation_size, sample_mean, sample_variance, generations = 100, p_cross = 0.7, best_frac = 0.5)
  batch_samples['sample_logl'] = disc_bn.logl(batch_samples)
  logl_mean = batch_samples['sample_logl'].mean(axis=0)
  logl_min = batch_samples['sample_logl'].min()
  logl_max = batch_samples['sample_logl'].max()

  # Gen readjustment
  mod_samples = batch_samples.copy()
  mod_samples['logl_mod'] = mod_samples.apply(lambda row : functions.logl_mod3(row['sample_logl'], threshold=logl_threshold, rep_attr_factor=logl_rep_attr), axis = 1)
  # mod_samples['logl_mod'] = mod_samples.apply(lambda row : functions.logl_mod4(row['sample_logl'], logl_min = logl_min, logl_max = logl_max, rep_attr_factor=logl_rep_attr), axis = 1)
  
  for nnode in noise_nodes:
    mod_samples[nnode] = mod_samples[nnode]*mod_samples['logl_mod']

  # gen_bn = functions.reset_gen(noise_nodes, base_nodes, noise_arcs)
  # gen_bn.fit(mod_samples)

  # gen_bn = functions.learn_condfromnet(learn_data = mod_samples[gen_nodes], interface_nodes = noise_nodes, nodes = base_nodes, forced_arcs = noise_arcs)
  gen_bn = functions.learn_condfromnet(learn_data = mod_samples[gen_nodes], interface_nodes = noise_nodes, nodes = base_nodes, banned_arcs = banned_arcs, banned_edges = banned_edges)
  assert(gen_bn.fitted())
  
  logl_array.append(logl_mean)
  
  pbar.update(1)

pbar.close()

##### 4.1. Gen testing

In [ ]:
# Logl evolution
plt.plot(logl_array, label="Logl")
plt.xlabel('Epoch')
plt.ylabel('Log-likelihood')
plt.plot([], [], ' ', label='Start logl: ' + str(round(logl_array[0],4)))
plt.plot([], [], ' ', label='End logl: ' + str(round(logl_array[-1],4)))
plt.legend()
plt.show()

# Learnt CPDs
print('Learnt CPDs:')
for node in base_nodes:
  print(gen_bn.cpd(node))

### 5. Anomaly detection

In [ ]:
# Reversed noise arcs
noise_arcs_reversed = []
for nnode in noise_nodes:
  for bnode in base_nodes:
    noise_arcs_reversed.append((bnode,nnode))

# Generating data for rgen training
reverse_data = functions.gen_samples(gen_bn, rgen_train_size, sample_mean, sample_variance)

# Learning reverse generator
# Banned arcs
reversed_banned_arcs = []
for bnode in base_nodes:
  for nnode in noise_nodes:
    reversed_banned_arcs.append((nnode,bnode))
rgen_bn = functions.learn_condnet(learn_data = reverse_data, interface_nodes = base_nodes, nodes = noise_nodes)
assert(rgen_bn.fitted())

for node in noise_nodes:
  print(rgen_bn.cpd(node))

##### 5.1. By anomaly score

In [ ]:
test_data = x_test.copy()

for i in range(mean_iter):
    noise_sample = rgen_bn.sample(evidence = test_data, concat_evidence = True, ordered = True)
    noise_sample = noise_sample.to_pandas()
    if i == 0:
        ano_score = noise_sample.apply(lambda row : functions.ano_score(row, noise_nodes, power = ano_power), axis = 1)
    else:
        ano_score = noise_sample.apply(lambda row : functions.ano_score(row, noise_nodes, power = ano_power), axis = 1) + ano_score

mean_ano_score = ano_score/mean_iter
test_data['ano_score'] = mean_ano_score.to_numpy()

In [ ]:
y_true = np.array(test_data['class'])
y_score_ano = np.array(test_data['ano_score'])

fpr, tpr, thresholds = metrics.roc_curve(y_true, y_score_ano)
roc_auc = metrics.auc(fpr, tpr)

print('Area under the ROC curve: ' + str(roc_auc))
display = metrics.RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc, estimator_name='BayesGEN NoiseAnoScore')
display.plot()
plt.show()

##### 5.2. By sample reconstruction

In [ ]:
test_data = x_test.copy()
noise_sample = rgen_bn.sample(evidence = test_data, concat_evidence = True, ordered = True)
noise_sample = noise_sample.to_pandas()

reconstructed_sample = gen_bn.sample(evidence = noise_sample[noise_nodes], concat_evidence = True, ordered = True).to_pandas()

diff_sample = noise_sample.copy()
diff_sample[base_nodes] = noise_sample[base_nodes] - reconstructed_sample[base_nodes]
modulo = diff_sample.apply(lambda row : functions.euclidean_mod(row, base_nodes), axis = 1)
diff_sample['rec_error'] = modulo

y_true = np.array(diff_sample['class'])
y_score_rec = np.array(diff_sample['rec_error'])

fpr, tpr, thresholds = metrics.roc_curve(y_true, y_score_rec)
roc_auc = metrics.auc(fpr, tpr)

print('Area under the ROC curve: ' + str(roc_auc))
display = metrics.RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc, estimator_name='BayesGEN Rec. Error')
display.plot()
plt.show()

##### 5.3. Applying both

In [ ]:
y_score_comb = np.multiply(y_score_ano, y_score_rec)
fpr, tpr, thresholds = metrics.roc_curve(y_true, y_score_comb)
roc_auc = metrics.auc(fpr, tpr)

print('Area under the ROC curve: ' + str(roc_auc))
display = metrics.RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc, estimator_name='BayesGEN Comb. score')
display.plot()
plt.show()